In [1]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, NodesGroupPointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, If, ConditionalExplanation, CompositeExplanation

## Example search tree:

In [2]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
leaf31 = MinMaxNode('leaf21', score=1)
leaf32 = MinMaxNode('leaf22', score=1)

child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child3 = MinMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

## Framework definition

In [3]:
BooleanAdjective("leaf",
        definition = "node.is_leaf")

BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn")

PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("leaf"),

            explanation_if_true = Assumption("Leaf nodes have scores from the evaluation function"),

            explanation_if_false = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        ))

PointerAdjective("backtracing child",
        definition = "node.score_child",

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),

            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),

            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        ))

ComparisonAdjective("better", "score", ">")
PointerAdjective("siblings", definition = "node.parent.children")

MaxRankAdjective("best", "better", "siblings")
MinRankAdjective("worst", "better", "siblings") # worst= not better than any sibling

In [4]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjectives([
    BooleanAdjective("leaf",
        definition = "node.is_leaf"),


    PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("leaf"),
            explanation_if_true = Assumption("Leaf nodes have scores from the evaluation function"),
            explanation_if_false = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        )),


    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("backtracing child",
        definition = "node.score_child",
        # alternative definition for backtracing child:
        # min(node.children, key=lambda child: child.score),

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),
            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("siblings",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("best", "better", "siblings"),

    MinRankAdjective("worst", "better", "siblings"),
])

In [5]:
print(knowledgebase)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backtracing child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
[(assumption) Definition of "leaf" is "node.is_leaf" → node is leaf]

[[(assumption) Definition of "leaf" is "node.is_leaf" → node is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function
 ∨ [(assumption) Definition of "leaf" is "node.is_leaf" → ¬(node is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → node is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ Node ¬(better) than all nodes in siblings → node is worst]

 ∨ [(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(node is opp

In [6]:
explainer = ArgumentativeExplainer(knowledgebase)

explainer.set_tree_search_motivation(lambda root: root.children, "best")

## Set your settings

In [7]:
settings = {
            'explanation_depth': 3 ,
            'assumptions_verbosity': 'verbose',
            'repeat_explanations': False
        }

knowledgebase.configure_settings(settings)

## You can now explain nodes' properties

In [25]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "n

In [9]:
settings = {
            'explanation_depth': 2 ,
            'assumptions_verbosity': 'verbose',
            'repeat_explanations': True
        }

knowledgebase.configure_settings(settings)

In [10]:
print(explainer.explain_adjective(child1, "better", child2))

[(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have sc

In [11]:
print(explainer.explain_adjective(child1, "siblings"))

[(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]



In [12]:
knowledgebase.settings.repeat_explanations = True

In [13]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "n

In [14]:
knowledgebase.settings.explanation_depth = 1

In [15]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "n

In [16]:
knowledgebase.settings.explanation_depth = 3
knowledgebase.settings.repeat_explanations = False

In [17]:
print(explainer.explain_adjective(child1, "worst"))

[(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" i

In [18]:
print(explainer.explain_adjective(leaf11, "worst"))

[(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]



In [19]:
print(explainer.explain_adjective(child2, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child2 has siblings = ['child1', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child2 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child2 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf22 has siblings = ['leaf21']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "n

In [20]:
print(explainer.explain_adjective(root, "backtracing child"))

[[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move. ∧ [(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children

In [21]:
print(explainer.explain_adjective(root, "score"))

[[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(root is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move. ∧ [(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is 

In [22]:
knowledgebase.settings.explanation_depth = 15
knowledgebase.settings.repeat_explanations = False

In [23]:
print(explainer.explain_adjective(child1, "best"))

[(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → child1 has siblings = ['child2', 'child3']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "node.is_leaf" → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn" → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ [(assumption) Definition of "siblings" is "node.parent.children excluding node" → leaf11 has siblings = ['leaf12']]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score ∧ [[(assumption) Definition of "leaf" is "n

Track down that Not before the "Considering your definition of leaf"

In [24]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
